# Introduction

### Requirements
#### Globus Nexus Client

* git clone https://github.com/globusonline/python-nexus-client
* python setup.py install


#### Create a Globus Account
* https://www.globus.org/SignUp

#### Save Access Token to a File
* curl -k --user &lt;username&gt;  'https://nexus.api.globusonline.org/goauth/token?grant_type=client_credentials'
* Copy the "access_token" from the response into a file "globus-token"

# Background

Configure the following variables required to use the client: the server address, username, and the file in which your access token is stored. 

You can then create a Globus Nexus client object to interact with Globus Nexus via its REST APIs. 

In [ ]:
from nexus import GlobusOnlineRestClient

server = "nexus.api.globusonline.org" 
username = "INSERT USERNAME"
token_file = "/PATH_TO_TOKEN/globus-token"
tutorial_group_id = "50b6a29c-63ac-11e4-8062-22000ab68755"

In [ ]:
with open(token_file, 'r') as f:
    globus_token = f.read()

config ={"server": server,
        "client": username, 
        "goauth_token": globus_token, 
        "client_secret": None}
client = GlobusOnlineRestClient(config)

# Interacting with Globus identities and profiles

The following steps will show you how to interact with Globus identities and profiles. 
### Retrieve a user profile
The profile includes information such as full name, email address, and institution. Additionally, custom attributes can be associated with a Globus identity. 

In [ ]:
response, data = client.get_user(username)
print data["fullname"]

### Retreive other attributes of the user profile
For example, profiles include any linked public SSH keys. These keys are often used for automatic provisioning of user accounts.

In [ ]:
print list(data.iteritems())
print 
print data["ssh_pubkeys"]

### Update a user profile
Many profile attributes can be updated, a notable exception is the username.

In [ ]:
response, data = client.put_user(username, fullname="Jeff Smith")
print data["fullname"]

# Interacting with Globus Groups

The following steps will show you how to interact with Globus Groups. 

### Get the groups for a user
You can set filters on user roles (e.g., admin, manager, member) and memberships (active, invited, pending, suspended, rejected). Groups include information such as name and description, parent groups, and links to retrieve and manage memberships, policies, and email templates. 

In [ ]:
response, data = client.get_group_list(my_roles=["member"])
for group in data:
    print "%s (%s)" % (group["name"], group["id"])

### Retrieve an individual group
All Globus groups are uniquely identified by an id. Assuming the policies of the group permit visibility, you can retrieve a summary of the group's attributes, policies regarding membership and invitation as well as the requesting user's membership status and role via the unique id. 

In [ ]:
response, data = client.get_group_summary(tutorial_group_id)
print data["name"]

### Retrieve group members
Note: the policies associated with this group do not allow non-members to see other members. If you are not a member yet, you should see an error response. 

In [ ]:
response, data = client.get_group_members(tutorial_group_id)
print response

### Request to join the group 
Individual users can request to join groups (assuming group policies permit it). Administrators may also invite users to join groups via their username or email address. 

In [ ]:
response, data = client.post_membership(tutorial_group_id, usernames=[username])
print data

### Retreive the members of this group
After your membership has been approved, you should be able to view the other members of the group. 

In [ ]:
response, data = client.get_group_members(tutorial_group_id)
for member in data["members"]:
    print member["name"]

### Create a new group
Users may create and manage their own groups. The only required information when creating a group is a name. After creation, you can change group attributes, invite users, and modify the group policies. 

In [ ]:
response, data = client.post_group("%s's first group" % username)
created_group_id = data["id"]
print "Group created with id: %s" % created_group_id

### Invite other users to join this group 
You can invite users to join your group using their Globus identity name or email address. Try to invite a user using your email adress. 

In [ ]:
response, data = client.post_membership(created_group_id, emails=["myemailaddress"])
print data

# Interacting directly with the REST API

The Globus Nexus REST API provides additional capabilities (a superset) that aren't supported via the Python client.

### Retrieve a user profile via the REST API

In [ ]:
response, data = client._issue_rest_request('/users/%s' % username)
print data["fullname"]

### Retrieve any linked credentials
If your Globus account has linked credentials (e.g., X509 certificates or SSH keys), you can retrieve a list as follows. 

In [ ]:
response, data = client._issue_rest_request('/users/%s/credentials' % username)
print data

### Other REST API calls
Similarly, group operations can be called via the REST API. 

In [ ]:
response, data = client._issue_rest_request('/groups?my_roles=member')
for group in data:
    print group['name']

In [ ]:
response, data = client._issue_rest_request('/groups/%s/policies' % tutorial_group_id)
print "Memberships visible publicly: %s" % data["group_member_visibility"]["value"]["public"]["value"]